# AMT Loader

This notebook demonstrates the AMT loader functions from `specparser.amt.loader`.

## AMT File Format

AMT files are YAML files with a specific structure:
- `backtest`: Configuration values (aum, leverage)
- `amt`: Asset definitions keyed by name, each with an `Underlying` identifier
- Rule tables: Tables with `Columns` and `Rows` for classification rules

In [ ]:
# Setup: Imports
import pandas as pd
from specparser.amt import (
    load_amt,
    clear_cache,
    get_value,
    get_aum,
    get_leverage,
    get_asset,
    find_assets,
    assets,
    cached_assets,
    _iter_assets,
    asset_class,
    get_table,
    _compile_rules,
    _match_rules,
    asset_table,
    asset_group,
    table_to_rows,
    show_table,
)

# Data paths (relative to this notebook)
AMT_PATH = "../data/amt.yml"

---
## 1. File Loading & Caching

### `load_amt(path)`

Load an AMT YAML file and return the full parsed data. Results are cached.

In [ ]:
# Load the AMT file
data = load_amt(AMT_PATH)

print(f"Top-level keys: {list(data.keys())}")
print(f"Number of assets: {len(data['amt'])}")
print(f"First 10 asset names: {list(data['amt'].keys())[:10]}")

### `clear_cache()`

Clear the AMT file cache. Useful when the file has been modified.

In [ ]:
# Clear and reload to demonstrate caching
clear_cache()
data = load_amt(AMT_PATH)  # This reloads from disk
print("Cache cleared and file reloaded")

---
## 2. Value Access

### `get_value(path, key_path, default)`

Get a value from an AMT file by its dot-separated key path.

In [ ]:
# Get nested values using dot notation
aum = get_value(AMT_PATH, "backtest.aum")
leverage = get_value(AMT_PATH, "backtest.leverage")

print(f"backtest.aum = {aum}")
print(f"backtest.leverage = {leverage}")

In [ ]:
# Default value for missing keys
missing = get_value(AMT_PATH, "nonexistent.key", default="NOT FOUND")
print(f"Missing key with default: {missing}")

### `get_aum(path)` and `get_leverage(path)`

Convenience functions for common backtest values.

In [ ]:
print(f"AUM: {get_aum(AMT_PATH):,.0f}")
print(f"Leverage: {get_leverage(AMT_PATH)}")

---
## 3. Asset Queries

### `get_asset(path, underlying)`

Get asset data by its Underlying value.

In [ ]:
# Get a commodity asset
cl = get_asset(AMT_PATH, "CL Comdty")

print("CL Comdty asset data:")
for key, value in cl.items():
    print(f"  {key}: {value}")

In [ ]:
# Get an equity index asset
spx = get_asset(AMT_PATH, "SPX Index")

print("SPX Index asset data:")
for key, value in spx.items():
    print(f"  {key}: {value}")

In [ ]:
# Get a currency asset
eurusd = get_asset(AMT_PATH, "EURUSD Curncy")

print("EURUSD Curncy asset data:")
for key, value in eurusd.items():
    print(f"  {key}: {value}")

In [ ]:
# Asset not found returns None
missing = get_asset(AMT_PATH, "NONEXISTENT")
print(f"Missing asset: {missing}")

### `find_assets(path, pattern, live_only)`

Find all Underlying values matching a regex pattern.

In [ ]:
# Find all equity assets
equities = find_assets(AMT_PATH, "Equity$")
print(f"Found {len(equities['rows'])} equity assets")
show_table(equities)

In [ ]:
# Find commodity assets that are live (WeightCap > 0)
live_commodities = find_assets(AMT_PATH, "Comdty$", live_only=True)
print(f"Live commodities (WeightCap > 0): {len(live_commodities['rows'])}")
show_table(live_commodities)

In [ ]:
# Find all index assets
indices = find_assets(AMT_PATH, "Index$", live_only=True)
print(f"Found {len(indices['rows'])} live index assets")
show_table(indices)

In [ ]:
# No matches returns empty table
no_match = find_assets(AMT_PATH, "^NOMATCH")
print(f"No matches: {len(no_match['rows'])} rows")
show_table(no_match)

### `assets(path, live_only, pattern)`

Get assets with their Underlying values.

In [ ]:
# All assets
all_assets = assets(AMT_PATH)
print(f"All assets: {len(all_assets['rows'])}")
show_table(all_assets)

In [ ]:
# Live assets only
live_assets = assets(AMT_PATH, live_only=True)
print(f"Live assets: {len(live_assets['rows'])}")
show_table(live_assets)

### `cached_assets(path)`

List all asset Underlying values from the cache.

In [ ]:
# Get assets from cache (faster, no regex matching)
cached = cached_assets(AMT_PATH)
print(f"Cached assets: {len(cached['rows'])}")
show_table(cached)

### `_iter_assets(path, live_only, pattern)`

Iterator over assets. Yields `(asset_data, underlying)` tuples.

In [ ]:
# Iterate over live commodity assets
print("Live commodity assets with their classes:")
for asset_data, underlying in _iter_assets(AMT_PATH, live_only=True, pattern="Comdty$"):
    print(f"  {underlying}: {asset_data.get('Class')}")

---
## 4. Asset Classification

### `asset_class(path, live_only, pattern)`

Get assets with their class, vol source, hedge source, and valuation model.

In [ ]:
# All live assets with classification info
classification = asset_class(AMT_PATH, live_only=True)
print(f"Asset classification: {len(classification['rows'])} rows")
show_table(classification)

In [ ]:
# Filter to specific pattern
commodity_classification = asset_class(AMT_PATH, live_only=True, pattern="Comdty$")
print(f"Commodity classification: {len(commodity_classification['rows'])} rows")
show_table(commodity_classification)

In [ ]:
# Analyze class distribution
from collections import Counter

all_class = asset_class(AMT_PATH, live_only=True)
classes = Counter(row[1] for row in all_class["rows"])  # col 1 is 'cls'

print("Asset class distribution:")
for cls, count in classes.most_common():
    print(f"  {cls}: {count}")

---
## 5. Embedded Tables

### `get_table(path, key_path)`

Get an embedded table from an AMT file. Tables have `Columns` and `Rows` keys.

In [ ]:
# Get the group_table if it exists
try:
    group_rules = get_table(AMT_PATH, "group_table")
    print(f"Columns: {group_rules['columns']}")
    show_table(group_rules)
except ValueError as e:
    print(f"Table not found: {e}")

In [ ]:
# Error case: table not found
try:
    get_table(AMT_PATH, "nonexistent_table")
except ValueError as e:
    print(f"Error: {e}")

---
## 6. Rule Matching

### `_compile_rules(table)`

Compile rules from a table with columns `[field, rgx, value]`.

In [ ]:
# Create a sample rules table
sample_rules = {
    "columns": ["field", "rgx", "value"],
    "rows": [
        ["Class", "^Equity$", "equities"],
        ["Class", "^Commodity$", "commodities"],
        ["Class", "^Currency$", "currencies"],
        ["Class", "^Rate$", "rates"],
        ["Class", ".*", "other"],
    ]
}

# Compile the rules
rules = _compile_rules(sample_rules)

print("Compiled rules (field, pattern, value):")
for field, pattern, value in rules:
    print(f"  {field}: {pattern.pattern!r} -> {value!r}")

### `_match_rules(rules, field_values, default)`

Find the first matching rule and return its value.

In [ ]:
# Match against compiled rules
result1 = _match_rules(rules, {"Class": "Equity", "Underlying": "AAPL US Equity"})
result2 = _match_rules(rules, {"Class": "Commodity", "Underlying": "CL Comdty"})
result3 = _match_rules(rules, {"Class": "Unknown", "Underlying": "XXX"})

print(f"Equity -> {result1}")
print(f"Commodity -> {result2}")
print(f"Unknown -> {result3} (caught by .* fallback)")

### `asset_table(path, table_name, default, live_only, pattern)`

Evaluate a classification rule table against assets.

In [ ]:
# Evaluate group_table against live assets (if it exists)
try:
    groups = asset_table(AMT_PATH, "group_table", live_only=True)
    show_table(groups)
except ValueError as e:
    print(f"group_table not found: {e}")

---
## 7. Asset Grouping

### `asset_group(path, live_only, pattern)`

Get assets with group, subgroup, liquidity, and limit override columns.

This combines multiple rule tables into one result.

In [ ]:
# Full asset grouping for live assets
full_grouping = asset_group(AMT_PATH, live_only=True)
print(f"Asset grouping: {len(full_grouping['rows'])} rows")
show_table(full_grouping)

In [ ]:
# Filter to specific pattern
commodity_grouping = asset_group(AMT_PATH, live_only=True, pattern="Comdty$")
print(f"Commodity grouping: {len(commodity_grouping['rows'])} rows")
show_table(commodity_grouping)

---
## 8. Practical Examples

### Example 1: Analyze Vol Sources by Asset Class

In [ ]:
# Get classification for all live assets
all_class = asset_class(AMT_PATH, live_only=True)

# Group by class and vol source
from collections import defaultdict

vol_by_class = defaultdict(Counter)
for row in all_class["rows"]:
    asset, cls, vol_source, hedge_source, model = row
    vol_by_class[cls][vol_source] += 1

print("Vol sources by asset class:")
for cls in sorted(vol_by_class.keys()):
    print(f"\n{cls}:")
    for source, count in vol_by_class[cls].most_common():
        print(f"  {source}: {count}")

### Example 2: Find Assets with Specific Configurations

In [ ]:
# Find assets using a specific vol source
all_class = asset_class(AMT_PATH, live_only=True)

# Filter to assets using BBG vol source
bbg_vol_assets = [row[0] for row in all_class["rows"] if row[2] == "BBG"]
print(f"Assets using BBG vol source: {len(bbg_vol_assets)}")
print(f"First 10: {bbg_vol_assets[:10]}")

In [ ]:
# Find assets with ES valuation model
es_model_assets = [row[0] for row in all_class["rows"] if row[4] == "ES"]
print(f"Assets using ES valuation model: {len(es_model_assets)}")
print(f"First 10: {es_model_assets[:10]}")

### Example 3: Asset Statistics

In [ ]:
# Count live vs non-live assets
all_assets = assets(AMT_PATH, live_only=False)
live_assets = assets(AMT_PATH, live_only=True)

print(f"Total assets: {len(all_assets['rows'])}")
print(f"Live assets: {len(live_assets['rows'])}")
print(f"Non-live assets: {len(all_assets['rows']) - len(live_assets['rows'])}")

In [ ]:
# Assets by suffix (Comdty, Index, Curncy, Equity)
suffixes = ["Comdty", "Index", "Curncy", "Equity"]

print("Live assets by type:")
for suffix in suffixes:
    matching = find_assets(AMT_PATH, f"{suffix}$", live_only=True)
    print(f"  {suffix}: {len(matching['rows'])}")

---
## 9. Summary

| Function | Description |
|----------|-------------|
| `load_amt` | Load and cache AMT YAML file |
| `clear_cache` | Clear the file cache |
| `get_value` | Get value by dot-separated key path |
| `get_aum` | Get AUM from backtest section |
| `get_leverage` | Get leverage from backtest section |
| `get_asset` | Get single asset by Underlying |
| `find_assets` | Find assets by regex pattern |
| `assets` | List all assets |
| `cached_assets` | List assets from cache |
| `_iter_assets` | Iterator over assets |
| `asset_class` | Assets with class/source info |
| `get_table` | Get embedded table by key path |
| `_compile_rules` | Compile regex rules from table |
| `_match_rules` | Match field values against rules |
| `asset_table` | Evaluate rule table against assets |
| `asset_group` | Assets with full grouping info |